In [1]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from feature_engineering import discretization as dc

# plt.style.use('seaborn-colorblind')
# %matplotlib inline
#from feature_cleaning import rare_values as ra

ModuleNotFoundError: ignored

## Load Dataset

In [ ]:
use_cols = [
    'Pclass', 'Sex', 'Age', 'Fare', 'SibSp',
    'Survived'
]

data = pd.read_csv('https://raw.githubusercontent.com/Yimeng-Zhang/feature-engineering-and-feature-selection/master/data/titanic.csv', usecols=use_cols)


In [ ]:
data.head(3)

In [ ]:
# Note that we include target variable in the X_train 
# because we need it to supervise our discretization
# this is not the standard way of using train-test-split
X_train, X_test, y_train, y_test = train_test_split(data, data.Survived, test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

## Equal width binning
divides the scope of possible values into N bins of the same width

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
enc_equal_width = KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='uniform').fit(X_train[['Fare']])

In [ ]:
# equal width for every bins
enc_equal_width.bin_edges_

In [ ]:
result = enc_equal_width.transform(X_train[['Fare']])
pd.DataFrame(result)[0].value_counts()

In [ ]:
# add the new discretized variable
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_equal_width'] = enc_equal_width.transform(X_train[['Fare']])
print(X_train_copy.head(10))

## Equal frequency binning
divides the scope of possible values of the variable into N bins, 
where each bin carries the same amount of observations

In [ ]:
enc_equal_freq = KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='quantile').fit(X_train[['Fare']])

In [ ]:
# check the bin edges
enc_equal_freq.bin_edges_

In [ ]:
# equal number of case for every bins
result = enc_equal_freq.transform(X_train[['Fare']])
pd.DataFrame(result)[0].value_counts()

In [ ]:
# add the new discretized variable
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_equal_freq'] = enc_equal_freq.transform(X_train[['Fare']])
print(X_train_copy.head(10))

## K-means binning
using k-means to partition values into clusters

In [ ]:
enc_kmeans = KBinsDiscretizer(n_bins=3,encode='ordinal',strategy='kmeans').fit(X_train[['Fare']])

In [ ]:
# check the bin edges
enc_kmeans.bin_edges_

In [ ]:
result = enc_kmeans.transform(X_train[['Fare']])
pd.DataFrame(result)[0].value_counts()

In [ ]:
# add the new discretized variable
X_train_copy = X_train.copy(deep=True)
X_train_copy['Fare_kmeans'] = enc_kmeans.transform(X_train[['Fare']])
print(X_train_copy.head(10))

## Discretisation with Decision Tree
using a decision tree to identify the optimal splitting points that would determine the bins

In [ ]:
enc1 = dc.DiscretizeByDecisionTree(col='Fare',max_depth=2).fit(X=X_train,y=y_train)

In [ ]:
enc1.tree_model

In [ ]:
data1 = enc1.transform(data)

In [ ]:
# see how the new column Fare_tree_discret is distributed
# the values are corresponding to the proba of the prediction by the tree
print(data1.head(5))

# the unique value of the discretisized column
print(data1.Fare_tree_discret.unique())

In [ ]:
# see how the bins are cut
# because we use a tree with max-depth of 2, we have at most 2*2=4 bins generated by the tree
col='Fare'
bins = pd.concat([data1.groupby([col+'_tree_discret'])[col].min(),
                  data1.groupby([col+'_tree_discret'])[col].max()], axis=1)
print(bins)

# all values between 0 to 7.5208 in the original variable 'Fare' 
# are given new value 0.107143 in the new column 'Fare_tree_discret'
# and so on

## Discretisation with Decision Tree with optimal depth search

In [ ]:
# search for the best depth from range 2-7
# we see when depth=2 we get the best roc-auc mean
enc2 = dc.DiscretizeByDecisionTree(col='Fare',max_depth=[2,3,4,5,6,7]).fit(X=X_train,y=y_train)

In [ ]:
# using optimal depth=2 we train the model, same result as last one
enc2.tree_model

In [ ]:
data2 = enc2.transform(data)
data2.head(5)

## Discretisation with ChiMerge
supervised hierarchical bottom-up (merge) method that locally exploits the chi-square criterion to decide whether two adjacent intervals are similar enough to be merged

In [ ]:
enc3 = dc.ChiMerge(col='Fare',num_of_bins=5).fit(X=X_train,y='Survived')

In [ ]:
# the bins boundary created by ChiMerge

enc3.bins

In [ ]:
data3 = enc3.transform(data)

In [ ]:
print(data3.head(5))

In [ ]:
# all values are grouped into 5 intervals
data3.Fare_chimerge.unique()